In [1]:
# ライブラリ(Selenium)のインポート
from selenium import webdriver
from time import sleep

# ライブラリ(Pandas)のインポート
import pandas as pd
from datetime import datetime as dt

# ドライバのロード
path = 'C:\\Users\\Owner\\chromedriver'
driver = webdriver.Chrome(path)

# 指定サイトのオープン
url = 'https://finance.yahoo.co.jp/quote/6701.T/history'
driver.get(url)

# メインのデータフレームに最初のページのデータ取得
data = pd.read_html(driver.current_url, header=0)
main = data[0]

# 画面操作ならびにデータ取得
try:
    # ボタンが押下可能な限りループ処理
    while True:
        # ボタン取得
        path = '//*[@id="pagerbtm"]/ul/li[7]/button'
        btn = driver.find_element_by_xpath(path)
        
        # 最終ページでボタンが押下不可か判定
        if btn.get_attribute('disabled') != None:
            break
        else:
            # 画面全高(100%)の25%程までスクロール
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.25);')
            sleep(1)
            # 次画面遷移
            btn.click()
            sleep(1)
            # データ取得と結合
            data = pd.read_html(driver.current_url, header=0)
            main = main.append(data[0], ignore_index=True)
except Exception:
    print('例外発生')

# 欠損値(NaN)を削除
main.dropna(inplace=True)

# 日付型のフィールド「修正日付」を追加
main['修正日付'] = [dt.strptime(val, "%Y年%m月%d日") for val in main['日付']]

# 不要なフィールド「日付」を削除
main.drop('日付', axis=1, inplace=True)

# フィールド「修正日付」をインデックス指定
main.set_index('修正日付', inplace=True)

# フィールド「修正日付」をキーに昇順にソート
main.sort_index(ascending=True, inplace=True)

In [2]:
# 終値が6250以上、6300未満のデータ表示
main[(main['終値'] >= 6250) & (main['終値'] < 6300)]

,始値,高値,安値,終値,出来高,調整後終値*
修正日付,,,,,,
2021-09-24,6170,6280,6170,6270,1135200,6270
2021-09-27,6270,6340,6250,6280,747000,6280
2021-10-15,6130,6280,6110,6270,1274100,6270
2021-10-18,6260,6290,6230,6280,751400,6280
2021-10-19,6300,6330,6260,6260,618700,6260
2021-10-22,6210,6280,6200,6250,657400,6250
2021-10-26,6250,6310,6230,6290,645500,6290


In [3]:
# 終値が6250以上、6300未満のデータをCSV出力
main[(main['終値'] >= 6250) & (main['終値'] < 6300)].to_csv(r'C:\\Users\\Owner\\Pandas-Exercises4\\6701_NEC.csv')

In [4]:
# CSVファイルからデータ読込
reader = pd.read_csv(r'C:\\Users\\Owner\\Pandas-Exercises4\\6701_NEC.csv', index_col=['修正日付'])

In [5]:
# 読込データ表示
reader

,始値,高値,安値,終値,出来高,調整後終値*
修正日付,,,,,,
2021-09-24,6170,6280,6170,6270,1135200,6270
2021-09-27,6270,6340,6250,6280,747000,6280
2021-10-15,6130,6280,6110,6270,1274100,6270
2021-10-18,6260,6290,6230,6280,751400,6280
2021-10-19,6300,6330,6260,6260,618700,6260
2021-10-22,6210,6280,6200,6250,657400,6250
2021-10-26,6250,6310,6230,6290,645500,6290
